In [ ]:
# 필요한 라이브러리 설치 및 임포트

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import warnings

from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

# 데이터 불러오기

In [ ]:
# 드라이브 사용 설정
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#  차주

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/kt_aivle/bigproject/data_preprocessing.csv', encoding='cp949')
df

,num,category,T,car_type,is_freezer,vibration-free,transport_type,Departure (1),Departure (2),Departure (3),Destination (1),Destination (2),Destination (3),Destination type
0,46640,3,3,1,0,2,3,11.0,166.0,송악읍,6.0,66.0,안영동,7.0
1,46669,3,3,1,0,2,2,8.0,111.0,팔탄면,8.0,111.0,동탄면,7.0
2,46673,3,6,1,0,2,3,9.0,119.0,동면,2.0,40.0,모라동,7.0
3,46676,5,3,1,0,2,3,6.0,67.0,관저동,4.0,50.0,북성동,6.0
4,46683,1,3,4,1,2,3,10.0,147.0,광혜원면,8.0,109.0,석정동,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4383,105113,3,6,1,0,2,3,3.0,45.0,봉덕동,3.0,45.0,대명동,7.0
4384,105249,5,3,1,0,2,2,11.0,151.0,영성,1.0,24.0,가락,6.0
4385,105372,3,6,1,0,2,3,4.0,51.0,송림,4.0,51.0,송현,7.0
4386,105534,5,6,1,0,2,3,3.0,45.0,봉덕동,3.0,48.0,호산동,6.0


In [ ]:
len(df['num'].unique())

4388

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4388 entries, 0 to 4387
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   num               4388 non-null   int64  
 1   category          4388 non-null   int64  
 2   T                 4388 non-null   int64  
 3   car_type          4388 non-null   int64  
 4   is_freezer        4388 non-null   int64  
 5   vibration-free    4388 non-null   int64  
 6   transport_type    4388 non-null   int64  
 7   Departure (1)     4388 non-null   float64
 8   Departure (2)     4388 non-null   float64
 9   Departure (3)     4388 non-null   object 
 10  Destination (1)   4388 non-null   float64
 11  Destination (2)   4388 non-null   float64
 12  Destination (3)   4388 non-null   object 
 13  Destination type  4388 non-null   float64
dtypes: float64(5), int64(7), object(2)
memory usage: 480.1+ KB


In [ ]:
df['Departure (1)'] = df['Departure (1)'].astype(int)
df['Departure (2)'] = df['Departure (2)'].astype(int)
df['Destination (1)'] = df['Destination (1)'].astype(int)
df['Destination (2)'] = df['Destination (2)'].astype(int)

# 지오코딩 위도경도

In [ ]:
Departure = {1:'서울특별시',
2:'부산광역시',
3:'대구광역시',
4:'인천광역시',
5:'광주광역시',
6:'대전광역시',
7:'울산광역시',
8:'경기도',
9:'강원도',
10:'충청북도',
11:'충청남도',
12:'전라북도',
13:'전라남도',
14:'경상북도',
15:'경상남도',
16:'제주특별자치도',
17:'세종특별자치시'}

In [ ]:
df['Destination (1)'] = df['Destination (1)'].replace(Departure)
df['Departure (1)'] = df['Departure (1)'].replace(Departure)
df.head()

,num,category,T,car_type,is_freezer,vibration-free,transport_type,Departure (1),Departure (2),Departure (3),Destination (1),Destination (2),Destination (3),Destination type
0,46640,3,3,1,0,2,3,충청남도,166,송악읍,대전광역시,66,안영동,7.0
1,46669,3,3,1,0,2,2,경기도,111,팔탄면,경기도,111,동탄면,7.0
2,46673,3,6,1,0,2,3,강원도,119,동면,부산광역시,40,모라동,7.0
3,46676,5,3,1,0,2,3,대전광역시,67,관저동,인천광역시,50,북성동,6.0
4,46683,1,3,4,1,2,3,충청북도,147,광혜원면,경기도,109,석정동,6.0


In [ ]:
Departure2={1:'종로구',
2:'중구',
3:'용산구',
4:'성동구',
5:'광진구',
6:'동대문구',
7:'중랑구',
8:'성북구',
9:'강북구',
10:'도봉구',
11:'노원구',
12:'은평구',
13:'서대문구',
14:'마포구',
15:'양천구',
16:'강서구',
17:'구로구',
18:'금천구',
19:'영등포구',
20:'동작구',
21:'관악구',
22:'서초구',
23:'강남구',
24:'송파구',
25:'강동구',
26:'중구',
27:'서구',
28:'동구',
29:'영도구',
30:'부산진구',
31:'동래구',
32:'남구',
33:'북구',
34:'해운대구',
35:'사하구',
36:'금정구',
37:'강서구',
38:'연제구',
39:'수영구',
40:'사상구',
41:'기장군',
42:'중구',
43:'동구',
44:'서구',
45:'남구',
46:'북구',
47:'수성구',
48:'달서구',
49:'달성군',
50:'중구',
51:'동구',
52:'남구',
53:'연수구',
54:'남동구',
55:'부평구',
56:'계양구',
57:'서구',
58:'강화군',
59:'옹진군',
60:'동구',
61:'서구',
62:'남구',
63:'북구',
64:'광산구',
65:'동구',
66:'중구',
67:'서구',
68:'유성구',
69:'대덕구',
70:'중구',
71:'남구',
72:'동구',
73:'북구',
74:'울주군',
75:'수원시 장안구',
76:'수원시 권선구',
77:'수원시 팔달구',
78:'수원시 영통구',
79:'성남시 수정구',
80:'성남시 중원구',
81:'성남시 분당구',
82:'의정부시',
83:'안양시 만안구',
84:'안양시 동안구',
85:'부천시 원미구',
86:'부천시 소사구',
87:'부천시 오정구',
88:'광명시',
89:'평택시',
90:'동두천시',
91:'안산시 상록구',
92:'안산시 단원구',
93:'고양시 덕양구',
94:'고양시 일산동구',
95:'고양시 일산서구',
96:'과천시',
97:'구리시',
98:'남양주시',
99:'오산시',
100:'시흥시',
101:'군포시',
102:'의왕시',
103:'하남시',
104:'용인시 처인구',
105:'용인시 기흥구',
106:'용인시 수지구',
107:'파주시',
108:'이천시',
109:'안성시',
110:'김포시',
111:'화성시',
112:'광주시',
113:'양주시',
114:'포천시',
115:'여주군',
116:'연천군',
117:'가평군',
118:'양평군',
119:'춘천시',
120:'원주시',
121:'강릉시',
122:'동해시',
123:'태백시',
124:'속초시',
125:'삼척시',
126:'홍천군',
127:'횡성군',
128:'영월군',
129:'평창군',
130:'정선군',
131:'철원군',
132:'화천군',
133:'양구군',
134:'인제군',
135:'고성군',
136:'양양군',
137:'청주시 상당구',
138:'청주시 흥덕구',
139:'청주시 청원구',
140:'청주시 서원구',
141:'충주시',
142:'제천시',
143:'보은군',
144:'옥천군',
145:'영동군',
146:'증평군',
147:'진천군',
148:'괴산군',
149:'음성군',
150:'단양군',
151:'천안시 동남구',
152:'천안시 서북구',
153:'공주시',
154:'보령시',
155:'아산시',
156:'서산시',
157:'논산시',
158:'계룡시',
159:'금산군',
160:'부여군',
161:'서천군',
162:'청양군',
163:'홍성군',
164:'예산군',
165:'태안군',
166:'당진군',
167:'전주시 완산구',
168:'전주시 덕진구',
169:'군산시',
170:'익산시',
171:'정읍시',
172:'남원시',
173:'김제시',
174:'완주군',
175:'진안군',
176:'무주군',
177:'장수군',
178:'임실군',
179:'순창군',
180:'고창군',
181:'부안군',
182:'목포시',
183:'여수시',
184:'순천시',
185:'나주시',
186:'광양시',
187:'담양군',
188:'곡성군',
189:'구례군',
190:'고흥군',
191:'보성군',
192:'화순군',
193:'장흥군',
194:'강진군',
195:'해남군',
196:'영암군',
197:'무안군',
198:'함평군',
199:'영광군',
200:'장성군',
201:'완도군',
202:'진도군',
203:'신안군',
204:'포항시 남구',
205:'포항시 북구',
206:'경주시',
207:'김천시',
208:'안동시',
209:'구미시',
210:'영주시',
211:'영천시',
212:'상주시',
213:'문경시',
214:'경산시',
215:'군위군',
216:'의성군',
217:'청송군',
218:'영양군',
219:'영덕군',
220:'청도군',
221:'고령군',
222:'성주군',
223:'칠곡군',
224:'예천군',
225:'봉화군',
226:'울진군',
227:'울릉군',
228:'창원시 의창구',
229:'창원시 성산구',
230:'창원시 마산합포구',
231:'창원시 마산회원구',
232:'창원시 진해구',
233:'진주시',
234:'통영시',
235:'사천시',
236:'김해시',
237:'밀양시',
238:'거제시',
239:'양산시',
240:'의령군',
241:'함안군',
242:'창녕군',
243:'고성군',
244:'남해군',
245:'하동군',
246:'산청군',
247:'함양군',
248:'거창군',
249:'합천군',
250:'제주시',
251:'서귀포시',
252:'세종시'}

In [ ]:
df['Destination (2)'] = df['Destination (2)'].replace(Departure2)
df['Departure (2)'] = df['Departure (2)'].replace(Departure2)
df.head()

,num,category,T,car_type,is_freezer,vibration-free,transport_type,Departure (1),Departure (2),Departure (3),Destination (1),Destination (2),Destination (3),Destination type
0,46640,3,3,1,0,2,3,충청남도,당진군,송악읍,대전광역시,중구,안영동,7.0
1,46669,3,3,1,0,2,2,경기도,화성시,팔탄면,경기도,화성시,동탄면,7.0
2,46673,3,6,1,0,2,3,강원도,춘천시,동면,부산광역시,사상구,모라동,7.0
3,46676,5,3,1,0,2,3,대전광역시,서구,관저동,인천광역시,중구,북성동,6.0
4,46683,1,3,4,1,2,3,충청북도,진천군,광혜원면,경기도,안성시,석정동,6.0


In [ ]:
df

,num,category,T,car_type,is_freezer,vibration-free,transport_type,Departure (1),Departure (2),Departure (3),Destination (1),Destination (2),Destination (3),Destination type
0,46640,3,3,1,0,2,3,충청남도,당진군,송악읍,대전광역시,중구,안영동,7.0
1,46669,3,3,1,0,2,2,경기도,화성시,팔탄면,경기도,화성시,동탄면,7.0
2,46673,3,6,1,0,2,3,강원도,춘천시,동면,부산광역시,사상구,모라동,7.0
3,46676,5,3,1,0,2,3,대전광역시,서구,관저동,인천광역시,중구,북성동,6.0
4,46683,1,3,4,1,2,3,충청북도,진천군,광혜원면,경기도,안성시,석정동,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4383,105113,3,6,1,0,2,3,대구광역시,남구,봉덕동,대구광역시,남구,대명동,7.0
4384,105249,5,3,1,0,2,2,충청남도,천안시 동남구,영성,서울특별시,송파구,가락,6.0
4385,105372,3,6,1,0,2,3,인천광역시,동구,송림,인천광역시,동구,송현,7.0
4386,105534,5,6,1,0,2,3,대구광역시,남구,봉덕동,대구광역시,달서구,호산동,6.0


In [ ]:
import pandas as pd
import requests

# 네이버 Geocoding API 정보
client_id = 'id 값'
client_secret = 'key 값'
api_url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode'

# 위도와 경도를 저장할 열 초기화
df['Departure_latitude'] = None
df['Departure_longitude'] = None
df['Destination_latitude'] = None
df['Destination_longitude'] = None

# Geocoding API 호출 및 위도, 경도 저장
def get_coordinates(df, index, departure_prefix, destination_prefix):
    departure_address = f"{df.at[index, departure_prefix + ' (1)']} {df.at[index, departure_prefix + ' (2)']} {df.at[index, departure_prefix + ' (3)']}"
    destination_address = f"{df.at[index, destination_prefix + ' (1)']} {df.at[index, destination_prefix + ' (2)']} {df.at[index, destination_prefix + ' (3)']}"

    for address, prefix in [(departure_address, 'Departure'), (destination_address, 'Destination')]:
        headers = {
            'X-NCP-APIGW-API-KEY-ID': client_id,
            'X-NCP-APIGW-API-KEY': client_secret
        }
        params = {'query': address}
        response = requests.get(api_url, headers=headers, params=params)
        result = response.json()

        if 'addresses' in result and result['addresses']:
            coordinates = result['addresses'][0]
            df.at[index, f'{prefix}_latitude'] = coordinates['y']
            df.at[index, f'{prefix}_longitude'] = coordinates['x']

# 출발지와 도착지에 대해 Geocoding API 호출
for index, row in df.iterrows():
    get_coordinates(df, index, 'Departure', 'Destination')

# 결과 출력
print(df)



         num  category  T  car_type  is_freezer  vibration-free  \
0      46640         3  3         1           0               2   
1      46669         3  3         1           0               2   
2      46673         3  6         1           0               2   
3      46676         5  3         1           0               2   
4      46683         1  3         4           1               2   
...      ...       ... ..       ...         ...             ...   
4383  105113         3  6         1           0               2   
4384  105249         5  3         1           0               2   
4385  105372         3  6         1           0               2   
4386  105534         5  6         1           0               2   
4387  105695         1  2         1           0               2   

      transport_type Departure (1) Departure (2) Departure (3)  \
0                  3          충청남도           당진군           송악읍   
1                  2           경기도           화성시           팔탄면 

In [ ]:
df

,num,category,T,car_type,is_freezer,vibration-free,transport_type,Departure (1),Departure (2),Departure (3),Destination (1),Destination (2),Destination (3),Destination type,Departure_latitude,Departure_longitude,Destination_latitude,Destination_longitude
0,46640,3,3,1,0,2,3,충청남도,당진군,송악읍,대전광역시,중구,안영동,7.0,36.904159,126.699458,36.2815,127.3778
1,46669,3,3,1,0,2,2,경기도,화성시,팔탄면,경기도,화성시,동탄면,7.0,37.162248,126.904961,None,None
2,46673,3,6,1,0,2,3,강원도,춘천시,동면,부산광역시,사상구,모라동,7.0,37.910276,127.781058,35.18698,128.989962
3,46676,5,3,1,0,2,3,대전광역시,서구,관저동,인천광역시,중구,북성동,6.0,36.303951,127.337361,37.475161,126.618491
4,46683,1,3,4,1,2,3,충청북도,진천군,광혜원면,경기도,안성시,석정동,6.0,36.993408,127.435584,37.0099,127.2594
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4383,105113,3,6,1,0,2,3,대구광역시,남구,봉덕동,대구광역시,남구,대명동,7.0,35.845294,128.596437,35.841747,128.573555
4384,105249,5,3,1,0,2,2,충청남도,천안시 동남구,영성,서울특별시,송파구,가락,6.0,36.800017,127.151625,None,None
4385,105372,3,6,1,0,2,3,인천광역시,동구,송림,인천광역시,동구,송현,7.0,None,None,37.486851,126.642225
4386,105534,5,6,1,0,2,3,대구광역시,남구,봉덕동,대구광역시,달서구,호산동,6.0,35.845294,128.596437,35.8537,128.4823


In [ ]:
df.to_csv('data_latlong.csv', index=False, encoding='cp949')

# 화주

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/kt_aivle/bigproject/factory_list_final.csv', encoding='cp949')
data

,공장주소_지번,생산품,업종명,용도지역
0,서울특별시 종로구 창성동 87-1번지,인쇄물,경 인쇄업 외 2 종,도시지역/주거지역/제2종일반주거지역/고도지구
1,서울특별시 종로구 통의동 35-69번지,"모형전시물,IC TESTER외",전시용 모형 제조업,도시지역/주거지역/주거지역/
2,서울특별시 종로구 통의동 35-6번지,주간신문,일반 서적 출판업 외 3 종,도시지역/주거지역/주거지역/
3,서울특별시 종로구 적선동 107-1번지,인쇄,경 인쇄업 외 2 종,도시지역/주거지역/제2종일반주거지역/미관지구
4,서울특별시 종로구 적선동 2번지,"정부간행물(책자,광고기획)",기타 인쇄업 외 1 종,도시지역/상업지역/일반상업지역/
...,...,...,...,...
197436,경상남도 합천군 대병면 대지리 843번지,쌀,곡물 도정업,관리지역/계획관리지역/계획관리지역/
197437,경상남도 합천군 용주면 장전리 627-2번지,화장품 제조업,화장품 제조업,관리지역/관리지역/관리지역/
197438,경상남도 합천군 용주면 방곡리 826-14번지,"플라스틱관, 레미콘, 콘크리트관","플라스틱 선, 봉, 관 및 호스 제조업 외 2 종",관리지역/관리지역/관리지역/
197439,경상남도 합천군 용주면 봉기리 33번지,톱밥,표면 가공 목재 및 특정 목적용 제재목 제조업,관리지역/계획관리지역/계획관리지역/


In [ ]:
data = data[data['업종명'].str.contains('광업|농림|축산|제조|건설|도소매|서비스')]
data

,공장주소_지번,생산품,업종명,용도지역
1,서울특별시 종로구 통의동 35-69번지,"모형전시물,IC TESTER외",전시용 모형 제조업,도시지역/주거지역/주거지역/
9,서울특별시 종로구 통인동 149-5번지,"가방, 지갑 등",가방 및 기타 보호용 케이스 제조업,도시지역/주거지역/제2종일반주거지역/고도지구
10,서울특별시 종로구 누하동 188번지,"간판, 광고물, 모형 제조",간판 및 광고물 제조업 외 1 종,도시지역/주거지역/제2종일반주거지역/지구단위1종수립지구
17,서울특별시 종로구 사직동 260번지,방범경호수신반외,방송장비 제조업 외 3 종,도시지역/주거지역/주거지역/
19,서울특별시 종로구 내수동 1번지,인터넷방송장비,유선 통신장비 제조업 외 1 종,도시지역/상업지역/일반상업지역/
...,...,...,...,...
197435,경상남도 합천군 대병면 장단리 649-1번지,장류제조,장류 제조업,농림지역/농업진흥지역/농업진흥구역/
197437,경상남도 합천군 용주면 장전리 627-2번지,화장품 제조업,화장품 제조업,관리지역/관리지역/관리지역/
197438,경상남도 합천군 용주면 방곡리 826-14번지,"플라스틱관, 레미콘, 콘크리트관","플라스틱 선, 봉, 관 및 호스 제조업 외 2 종",관리지역/관리지역/관리지역/
197439,경상남도 합천군 용주면 봉기리 33번지,톱밥,표면 가공 목재 및 특정 목적용 제재목 제조업,관리지역/계획관리지역/계획관리지역/


In [ ]:
# 랜덤으로 2만개의 데이터 추출
sampled_data = data.sample(n=20000, random_state=42)  # random_state는 재현성을 위한 시드 값입니다.

# 추출된 데이터 확인
sampled_data.head()

,공장주소_지번,생산품,업종명,용도지역
93139,경기도 파주시 광탄면 방축리 190번지,"배전반, 계장제어장치 등",공기 조화장치 제조업 외 6 종,관리지역/계획관리지역/계획관리지역/
89704,경기도 파주시 오도동 1-1번지,금속다이케스팅기,금속 주조 및 기타 야금용 기계 제조업,관리지역/계획관리지역/계획관리지역/
123484,경기도 여주시 천송동 497-3번지,도자기,가정용 및 장식용 도자기 제조업,자연환경보전지역/상수원보호구역/상수원보호구역/
82627,경기도 시흥시 정왕동 2101-8번지 (주)대진철구강업 시화단지 3마 508호,액체 여과기,액체 여과기 제조업,도시지역/공업지역/일반공업지역/
129667,충청북도 청주시 흥덕구 현암동 19-6번지,특수필름관련장비,산업처리공정 제어장비 제조업,도시지역/녹지지역/자연녹지지역/


In [ ]:
sampled_data = sampled_data.reset_index(drop=True)
sampled_data

,공장주소_지번,생산품,업종명,용도지역
0,경기도 파주시 광탄면 방축리 190번지,"배전반, 계장제어장치 등",공기 조화장치 제조업 외 6 종,관리지역/계획관리지역/계획관리지역/
1,경기도 파주시 오도동 1-1번지,금속다이케스팅기,금속 주조 및 기타 야금용 기계 제조업,관리지역/계획관리지역/계획관리지역/
2,경기도 여주시 천송동 497-3번지,도자기,가정용 및 장식용 도자기 제조업,자연환경보전지역/상수원보호구역/상수원보호구역/
3,경기도 시흥시 정왕동 2101-8번지 (주)대진철구강업 시화단지 3마 508호,액체 여과기,액체 여과기 제조업,도시지역/공업지역/일반공업지역/
4,충청북도 청주시 흥덕구 현암동 19-6번지,특수필름관련장비,산업처리공정 제어장비 제조업,도시지역/녹지지역/자연녹지지역/
...,...,...,...,...
19995,전라북도 김제시 백산면 부거리 1579-1번지,"양돈케이지, 환풍기",그 외 기타 플라스틱 제품 제조업 외 1 종,도시지역/공업지역/일반공업지역/
19996,경기도 포천시 내촌면 음현리 10-21,스폰지,그외 기타 플라스틱 제품 제조업,농림지역/준보전산지지역/준보전산지지/
19997,경기도 화성시 비봉면 양노리 669-5번지,차량용 냉동기기,산업용 냉장 및 냉동 장비 제조업 외 1 종,관리지역/계획관리지역/계획관리지역/
19998,경기도 용인시처인구 모현읍 초부리 46번지,건축용목제품,기타 건축용 나무제품 제조업 외 1 종,관리지역/관리지역/관리지역/


In [ ]:
from tqdm.notebook import tqdm  # 수정된 부분
import pandas as pd
import requests

# 네이버 Geocoding API 정보
client_id = 'id 값'
client_secret = 'key 값'
api_url = 'https://naveropenapi.apigw.ntruss.com/map-geocode/v2/geocode'

# 위도와 경도를 저장할 열 초기화
sampled_data['Departure_latitude'] = None
sampled_data['Departure_longitude'] = None

# Geocoding API 호출 및 위도, 경도 저장
def get_coordinates(sampled_data, index, departure_prefix):
    departure_address = f"{sampled_data.at[index, departure_prefix]}"

    for address, prefix in [(departure_address, 'Departure')]:
        headers = {
            'X-NCP-APIGW-API-KEY-ID': client_id,
            'X-NCP-APIGW-API-KEY': client_secret
        }
        params = {'query': address}
        response = requests.get(api_url, headers=headers, params=params)
        result = response.json()

        if 'addresses' in result and result['addresses']:
            coordinates = result['addresses'][0]
            sampled_data.at[index, f'{prefix}_latitude'] = coordinates['y']
            sampled_data.at[index, f'{prefix}_longitude'] = coordinates['x']

# 출발지와 도착지에 대해 Geocoding API 호출
for index, row in tqdm(sampled_data.iterrows(), total=len(sampled_data), desc="Geocoding 진행 중"):
    get_coordinates(sampled_data, index, '공장주소_지번')

# 결과 출력
sampled_data


Geocoding 진행 중:   0%|          | 0/20000 [00:00<?, ?it/s]

,공장주소_지번,생산품,업종명,용도지역,Departure_latitude,Departure_longitude
0,경기도 파주시 광탄면 방축리 190번지,"배전반, 계장제어장치 등",공기 조화장치 제조업 외 6 종,관리지역/계획관리지역/계획관리지역/,37.8141048,126.8668589
1,경기도 파주시 오도동 1-1번지,금속다이케스팅기,금속 주조 및 기타 야금용 기계 제조업,관리지역/계획관리지역/계획관리지역/,37.7504465,126.7358746
2,경기도 여주시 천송동 497-3번지,도자기,가정용 및 장식용 도자기 제조업,자연환경보전지역/상수원보호구역/상수원보호구역/,37.3037713,127.6499877
3,경기도 시흥시 정왕동 2101-8번지 (주)대진철구강업 시화단지 3마 508호,액체 여과기,액체 여과기 제조업,도시지역/공업지역/일반공업지역/,37.3204909,126.7283197
4,충청북도 청주시 흥덕구 현암동 19-6번지,특수필름관련장비,산업처리공정 제어장비 제조업,도시지역/녹지지역/자연녹지지역/,36.5893693,127.3954256
...,...,...,...,...,...,...
19995,전라북도 김제시 백산면 부거리 1579-1번지,"양돈케이지, 환풍기",그 외 기타 플라스틱 제품 제조업 외 1 종,도시지역/공업지역/일반공업지역/,35.8452453,126.8806582
19996,경기도 포천시 내촌면 음현리 10-21,스폰지,그외 기타 플라스틱 제품 제조업,농림지역/준보전산지지역/준보전산지지/,37.7770589,127.2411229
19997,경기도 화성시 비봉면 양노리 669-5번지,차량용 냉동기기,산업용 냉장 및 냉동 장비 제조업 외 1 종,관리지역/계획관리지역/계획관리지역/,37.2192408,126.8550750
19998,경기도 용인시처인구 모현읍 초부리 46번지,건축용목제품,기타 건축용 나무제품 제조업 외 1 종,관리지역/관리지역/관리지역/,37.3147860,127.2349822


In [ ]:
sampled_data.to_csv('data_owner.csv', index=False, encoding='cp949')